## Overview

Notebook demonstrates novel unsupervised ML workflow that works on concatenated 4D-STEM and STEM-EDS datasets.

# Imports

In [ ]:
%matplotlib qt5



import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

import hdbscan
import umap

import hyperspy.api as hs
import pyxem as pxm

## Supporting functions for visualization

In [36]:
def palette_for_label_map(no_cluster, unclassified=False, mask=False):
    """
    no_cluster = number of real clusters
    unclassified = no_cluster + 1
    mask = no_cluster + 2

    Returns palette, CustomCmap
    """
    import seaborn as sns
    import matplotlib

    custom_palette = [plt.cm.tab20(i) for i in range(no_cluster)]

    if unclassified == True:
        custom_palette.insert(0, "gray")  # UNCLASSIFIED
    if mask == True:
        custom_palette.insert(0, "black")  # MASKED OUT

    CustomCmap = matplotlib.colors.ListedColormap(custom_palette)
    palette = sns.color_palette(palette=custom_palette)
    return (palette, CustomCmap)

def include_scalebar(dp):
    from matplotlib_scalebar.scalebar import ScaleBar

    dx = dp.axes_manager[0].scale
    scalebar = ScaleBar(
        dx,
        "nm",
        length_fraction=0.25,
        width_fraction=0.015,
        location="lower left",
        frameon=False,
        color="w",
        scale_loc="top",
        border_pad=0.5,
    )
    plt.gca().add_artist(scalebar)


def plot_label_map(labels_highest_soft_spatial, no_cluster_soft, scalebar=False):
    palette, CustomCmap = palette_for_label_map(
        no_cluster_soft, unclassified=False, mask=False
    )

    plt.figure()
    plt.imshow(labels_highest_soft_spatial, cmap=CustomCmap)

    if scalebar == True:
        include_scalebar(dp)
    
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.show()

def plot_manifold_labels(no_cluster_soft,labels_highest_soft, reduced_data ):

    import seaborn as sns
    cluster_colors = [palette_for_label_map(no_cluster_soft)[0][x] if x >= 0
                    else (0.5, 0.5, 0.5)
                    for x in labels_highest_soft]
    if reduced_data.shape[1]==3:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.scatter(reduced_data[:,0], reduced_data[:,1], reduced_data[:,2], alpha=0.2,  c=cluster_colors)
    
    if reduced_data.shape[1]==2:
        fig = plt.figure()
        plt.scatter(reduced_data[:,0], reduced_data[:,1], alpha=0.2,  c=cluster_colors)


In [37]:
def dp_cluster_std_dev(selected_dp, center_x=128, center_y=128, radius=11):
    cluster_std_dev_dp_pixels = np.std(selected_dp, axis=0)
    x, y = np.meshgrid(
        np.arange(cluster_std_dev_dp_pixels.data.shape[0]),
        np.arange(cluster_std_dev_dp_pixels.data.shape[1]),
    )
    distances = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
    mask = distances <= radius

    cluster_std_dev_dp_pixels_masked = np.copy(cluster_std_dev_dp_pixels)
    cluster_std_dev_dp_pixels_masked[mask] = 0
    return cluster_std_dev_dp_pixels_masked


def get_cluster_mean_dp_list(dp, memberships_highest_soft, labels_highest_soft):
    dp.fold()
    dp.unfold_navigation_space()
    dp_soft_cluster_mean_list = []

    for i in range(no_cluster_soft):
        selected_dp = np.take(
            dp.data,
            np.where((memberships_highest_soft == 1) & (labels_highest_soft == i))[0],
            axis=0,
        )
        dp_soft_cluster_mean = np.mean(selected_dp, axis=0)
        dp_soft_cluster_mean_list.append(dp_soft_cluster_mean)
    return dp_soft_cluster_mean_list


def get_cluster_std_dev_dp_list(dp, memberships_highest_soft, labels_highest_soft):
    dp.fold()
    dp.unfold_navigation_space()
    dp_soft_cluster_std_dev_list = []
    for i in range(no_cluster_soft):
        selected_dp = np.take(
            dp.data,
            np.where((memberships_highest_soft == 1) & (labels_highest_soft == i))[0],
            axis=0,
        )
        dp_soft_cluster_std_dev_list.append(dp_cluster_std_dev(selected_dp))
    return dp_soft_cluster_std_dev_list


def get_cluster_mean_eds_list(eds, memberships_highest_soft, labels_highest_soft):
    eds.fold()
    eds.unfold_navigation_space()
    eds_soft_cluster_mean_list = []
    for i in range(no_cluster_soft):
        selected_eds = np.take(
            eds.data,
            np.where((memberships_highest_soft == 1) & (labels_highest_soft == i))[0],
            axis=0,
        )
        eds_soft_cluster_mean = np.mean(selected_eds, axis=0)
        eds_soft_cluster_mean_list.append(eds_soft_cluster_mean)
    return eds_soft_cluster_mean_list


def get_cluster_std_dev_eds_list(eds, memberships_highest_soft, labels_highest_soft):
    eds.fold()
    eds.unfold_navigation_space()
    eds_soft_cluster_std_dev_list = []
    for i in range(no_cluster_soft):
        selected_eds = np.take(
            eds.data,
            np.where((memberships_highest_soft == 1) & (labels_highest_soft == i))[0],
            axis=0,
        )
        eds_soft_cluster_std_dev = np.std(selected_eds, axis=0)
        eds_soft_cluster_std_dev_list.append(eds_soft_cluster_std_dev)
    return eds_soft_cluster_std_dev_list


def plot_cluster_mean_analysis(
    include_eds_mean_plot=False,
    include_eds_std_dev_plot=False,
    include_dp_std_dev_plot=False,
    include_dp_mean_plot=True,
    include_probability_map=True,
    dp_vmax=0.1,
):
    from matplotlib_scalebar.scalebar import ScaleBar

    palette, CustomCmap = palette_for_label_map(
        no_cluster_soft, unclassified=False, mask=False
    )

    # Calculate the total number of rows required based on the plots to be included
    total_rows = 0

    if include_dp_mean_plot:
        total_rows += 1

    if include_probability_map:
        total_rows += 2

    if include_dp_std_dev_plot:
        total_rows += 1

    if include_eds_mean_plot:
        total_rows += 1

    if include_eds_std_dev_plot:
        total_rows += 1

    fig = plt.figure(figsize=(7.08661, 7.08661))

    gs = plt.GridSpec(
        total_rows,
        no_cluster_soft,
        wspace=0.1,
        hspace=0.1,
        height_ratios=[1] * (total_rows - 1) + [0.1],
    )

    for i in range(no_cluster_soft):
        row_index = 0  # Initialize row index

        ax_outer = plt.subplot(gs[:, i])
        ax_outer.set_facecolor(palette[i])
        plt.xticks([])
        plt.yticks([])

        if include_dp_mean_plot:
            dp_soft_cluster_mean_list = get_cluster_mean_dp_list(
                dp, memberships_highest_soft, labels_highest_soft
            )

            ax_ebsd = fig.add_subplot(gs[row_index, i])
            dp_cluster_mean = dp_soft_cluster_mean_list[i]  # current cluster
            dp_cluster_mean_normalize = (dp_cluster_mean - dp_cluster_mean.min()) / (
                dp_cluster_mean.max() - dp_cluster_mean.min()
            )  # normalize
            plt.imshow(dp_cluster_mean_normalize, cmap="Greys_r", vmax=dp_vmax)
            plt.xticks([])
            plt.yticks([])

            row_index += 1

        if include_dp_std_dev_plot:
            dp_soft_cluster_std_dev_list = get_cluster_std_dev_dp_list(
                dp, memberships_highest_soft, labels_highest_soft
            )
            ax_dp_std = fig.add_subplot(gs[row_index, i])
            dp_cluster_std = dp_soft_cluster_std_dev_list[i]
            plt.imshow(dp_cluster_std, cmap="inferno")
            plt.xticks([])
            plt.yticks([])

            row_index += 1

        if include_eds_mean_plot:
            eds_soft_cluster_mean_list = get_cluster_mean_eds_list(
                eds, memberships_highest_soft, labels_highest_soft
            )
            ax_eds = fig.add_subplot(gs[row_index, i])
            eds_cluster_mean = eds_soft_cluster_mean_list[i]
            plt.plot(eds_cluster_mean)
            plt.xticks([])
            plt.yticks([])

            row_index += 1

        if include_eds_std_dev_plot:
            eds_soft_cluster_std_dev_list = get_cluster_std_dev_eds_list(
                eds, memberships_highest_soft, labels_highest_soft
            )
            ax_eds_std = fig.add_subplot(gs[row_index, i])
            eds_cluster_std = eds_soft_cluster_std_dev_list[i]
            plt.plot(eds_cluster_std, "g")
            plt.xticks([])
            plt.yticks([])

            row_index += 1

        if include_probability_map:
            ax_loading = fig.add_subplot(gs[row_index, i])
            memberships_highest_soft_spatial2 = memberships_highest_soft_spatial.copy()
            memberships_highest_soft_spatial2[
                np.where(labels_highest_soft_spatial != i)
            ] = 0
            pc = plt.imshow(memberships_highest_soft_spatial2)

            # scalebar = ScaleBar(2, 'nm', length_fraction=0.25, width_fraction=0.015, location='lower left',
            #             frameon=False, color='w', scale_loc='top', border_pad=0.1)  # 1 pixel = 0.2 meter
            # plt.gca().add_artist(scalebar)

            plt.xticks([])
            plt.yticks([])

            row_index += 1

    ax_colorbar = fig.add_subplot(gs[row_index, :])
    plt.colorbar(pc, cax=ax_colorbar, orientation="horizontal")


# Start of workflow

## Load data using hyperspy

In [2]:
dirpath = f"C:\\Users\\Zhi Quan\\Dropbox (Personal)\\Jupyter backup\\Heusler alloy\\Data\\06_15_22\\"

In [3]:
eds = hs.load(dirpath + 'Heusler-IA57-2-15-June__default8__EDS.hspy', lazy=False)
eds.set_signal_type("EDS_TEM")

dp = hs.load(dirpath + "Heusler-IA57-2-15-June__default8.hspy", lazy=False)
dp.change_dtype("float32")
dp

c:\ProgramData\Miniconda3\envs\datascience_3.9\lib\site-packages\hyperspy\io.py:650: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn(
c:\ProgramData\Miniconda3\envs\datascience_3.9\lib\site-packages\hyperspy\io.py:650: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn(


<Signal2D, title: , dimensions: (150, 50|256, 256)>

## Visualize data

In [5]:
dp.plot(norm='symlog')

In [6]:
eds.plot()

# Feature extraction

Binning is mostly for computational efficiency.

## 4D-STEM

In [7]:
dp_rebin = dp.rebin(scale=(1,1,2,2)) 
dp_rebin

<Signal2D, title: , dimensions: (150, 50|128, 128)>

In [8]:
dp_rebin_shape = dp_rebin.data.shape[2]
dp_rebin_shape

128

## Standard deviation thresholding 

The within-feature standard deviation of each pixel in reciprocal space is calculated across all observations in the log-scaled 4D-STEM dataset resulting in a standard deviation map in reciprocal space. 

Log-scaling has to be applied to equalize standard deviations between weaker
and stronger reflections.

A central spot mask is applied next as it often contains significant variations that are uncorrelated to structure. 

A percentile threshold value is then defined, of which pixels corresponding to standard deviations above the threshold are chosen as features in a 2D array.

In general, there is relatively high stability in results using threshold values between 60-80%. Ideally you would want to explore higher percentile thresholds as we are primarily interested in reducing the sparsity/background features present in the 4D-STEM dataset as much as possible.

This particular feature engineering technique works better with discrete Bragg spots, and a different form of feature engineering might be required for other systems - e.g. radial profile, annular average, cepstrum etc. It is recommended to try cepstrum next if SDT is not working well. 

In [ ]:
dp_rebin.fold()
dp_rebin.set_signal_type('diffraction')

In [20]:
dp_rebin.unfold()

std_dev_dp_pixels = np.std(dp_rebin.data, axis=0)
std_dev_dp_pixels = hs.signals.Signal2D(
    std_dev_dp_pixels.reshape(dp_rebin_shape, dp_rebin_shape)
) 

# Define finalized center coordinates and radius of the circular region of interest
center_x, center_y = (62, 60)
radius = 9

# Create a grid of pixel coordinates
x, y = np.meshgrid(
    np.arange(std_dev_dp_pixels.data.shape[0]),
    np.arange(std_dev_dp_pixels.data.shape[1]),
)

# Calculate the distance of each pixel to the center of the circle
distances = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)

# Create a boolean mask where True values correspond to pixels within the circle
mask = distances <= radius

# Apply the mask to the image
std_dev_dp_pixels_masked_central = np.copy(std_dev_dp_pixels.data)
std_dev_dp_pixels_masked_central[mask] = 0


std_dev_dp_pixels_masked_central = hs.signals.Signal2D(std_dev_dp_pixels_masked_central)
std_dev_dp_pixels_masked_central.plot(norm='symlog')

In [21]:
final_thres = 0.4

matrix = std_dev_dp_pixels_masked_central.data

n = int(np.ceil(matrix.size * final_thres))
# n=1024

# Flatten the matrix into a 1D array and sort it in descending order
sorted_array = np.sort(matrix.flatten())[::-1]

# Extract the highest n values from the sorted array
top_n = sorted_array[:n]

# Reshape the top_n array back into a 2D matrix
# Create a mask for the top n values
mask = np.isin(matrix, top_n)

# Set all values not in the top n to zero
result = matrix * mask


result = hs.signals.Signal2D(result)
result.plot(norm='symlog')

In [22]:
dp_rebin.fold()
dp_rebin.unfold_navigation_space()

extracted_pixels_list = []

for d in dp_rebin.data:
    extracted_pixels = d[np.where(mask==1)]
    extracted_pixels_list.append(extracted_pixels)

extracted_pixels_array = np.array(extracted_pixels_list)
extracted_pixels_array[extracted_pixels_array < 0] = 0
extracted_pixels_array_scaled = np.log(extracted_pixels_array, where=extracted_pixels_array != 0)

## Visualize dataset after feature extraction

In [23]:
hs.signals.Signal1D(extracted_pixels_array_scaled.reshape(50,150,-1)).plot()

## EDS

## Check per-channel variance plot

In [11]:
eds.unfold()
eds_var = np.var(eds.data, axis=0)
hs.signals.Signal1D(eds_var).plot()

## Extract channels with channel variance greater than a certain threshold.

Threshold is dependent on the dataset's background variance based on the plot above.

In addition, certain known physical artefacts such as Cu peaks from the TEM grids can be manually excluded even if they represent high variance.

In [12]:
eds.unfold()
extracted_channel_list = []

for spect in eds.data:
    extracted_channel = spect[np.where(eds_var>0.012)]
    extracted_channel_list.append(extracted_channel)

extracted_channel_array = np.array(extracted_channel_list)
extracted_channel_array.shape

(7500, 758)

In [13]:
eds.unfold()
extracted_channel_array_scaled = np.log1p(extracted_channel_array)

In [14]:
extracted_channel_array_scaled.shape

(7500, 758)

## Visualize dataset after feature extraction

In [15]:
hs.signals.Signal1D(extracted_channel_array_scaled.reshape(50,150,-1)).plot()

# Perform signal merging

## Scaling of EDS

The number of DP pixel features greatly outweigh the number of STEM-EDS
channel features. 

In addition, the majority of DP pixel features contain relatively high
intensity magnitudes, compared to the counts in the STEM-EDS channels.

This illustrates the non-trivial problem of the fusion of blocks of data that have different scales, sizes and underlying dimensionalities.

As such, a scaling factor is required to bias the STEM-EDS dataset further.

It is recommended to do a sensitivity analysis as required, although a general rule of thumb that worked well between the systems tested is to scale the STEM-EDS dataset such that the maximum value is twice that of the 4D-STEM dataset. 

In general, you would want to bias towards the EDS dataset, as DP intensity variations within the same spots can lead to clustering based on tilts/thickness effects. The inclusion of weighted STEM-EDS suppresses these effects. 

The `np.exp` function is purely to help explore a wider range of values in a more efficient manner.

It is important to include the inter-dataset scaler (RobustScaler) used in the later workflow as part of the sensitivity analysis as that directly affects the values of the features.

In [24]:
extracted_channel_array_scaled.shape, extracted_pixels_array_scaled.shape

((7500, 758), (7500, 6554))

In [29]:
scaling_factor = 2.4

np.max(RobustScaler(with_centering=True).fit_transform(extracted_channel_array_scaled*np.exp(scaling_factor))) # max value is twice that of DP

19.750880660985626

In [30]:
np.max(RobustScaler(with_centering=True).fit_transform(extracted_pixels_array_scaled*np.exp(0)))

10.307353

## Concatenate

In [33]:
combined_data = np.concatenate((extracted_pixels_array_scaled, extracted_channel_array_scaled*np.exp(scaling_factor)), axis=1)
combined_data = hs.signals.Signal1D(combined_data.reshape(50,150,-1))

In [34]:
combined_data.plot()

# Perform dimension reduction and clustering on merged datasets

Hyperparameter understanding from official documentation:

https://umap-learn.readthedocs.io/en/latest/parameters.html

https://hdbscan.readthedocs.io/en/latest/parameter_selection.html


UMAP:

`min_distance=0`, `densmap=True` and `n_neighbors=10` encourages compact and well-separated clusters and should generally not require changing based on previous experience. In general, higher values of `min_distance` and `n_neighbors` would encourage a relatively more continuous distribution of points in latent space as compared to compact and well-separated clusters. This may be necessary to change when considering more continuous changes such as diffusion processes.

`n_components` generally produces stable cluster results across a wide range of components, and should not require changing even if the system complexity increases. This is unlike linear dimension reduction techniques, where additional components can drastically alter the microstructure component resulting from the linear addition of an entirely new phase. 

`metric` the distance metric used to compute the latent space. A more general starting metric to try would be `eucldiean`. `chebyshev` identifies the maximally dissimilar feature between two points and uses that to compute the distance. `euclidean` averages dissimilarity in all features between two points.

HDBSCAN:

`min_cluster_size` this was shown to be a relatively intuitive parameter that relates to the smallest microstrucrture feature present in the scan in terms of the number of pixels, which can be achieved just through the inverse 4D-STEM plot using Hyperspy `dp.T.plot()` or the HAADF associated with the STEM-EDS dataset.

`min_samples` is not an important parameter in the context of soft implementation of HDBSCAN, which models points as part of multiple clusters instead of modelling the probability that a point belongs to a hard cluster label. The parameter controls how conservative it is in assigning points to clusters vs outliers.

In [56]:
from sklearn.preprocessing import RobustScaler

combined_data.unfold()

pipe = Pipeline([
        ('scale', RobustScaler(with_centering=True)),
        ('reduce_dims', umap.UMAP(densmap=True, n_neighbors=10, n_components=3, min_dist=0, metric='euclidean')),
        ])

reduced_data = pipe.fit_transform(combined_data.data)

clust = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=1, prediction_data=True)

clust.fit(reduced_data)

x,y = 50,150

memberships_all_soft = hdbscan.all_points_membership_vectors(clust)
memberships_highest_soft = np.array([np.max(x) for x in memberships_all_soft])
memberships_highest_soft_spatial = memberships_highest_soft.reshape(x, y)
labels_highest_soft = np.array([np.argmax(x) for x in memberships_all_soft])
labels_highest_soft_spatial = labels_highest_soft.reshape(x, y)
no_cluster_soft = len(set(labels_highest_soft))


## Visualize cluster results

In [57]:
plot_label_map(labels_highest_soft_spatial,no_cluster_soft) 

In [ ]:
plot_cluster_mean_analysis(include_eds_spectra=True)
# Optional parameters allow plotting for associated EDS spectra, the per cluster standard deviations
# The standard deviation maps in particular are helpful in assessing whether the clusters produced are accurate.

# Check stability of results

The supplementary materials include a novel stability metric that identifies the stability of cluster results between different runs by producing a cluster stability map. For exploratory analysis, just re-running the workflow with the same parameters will give a good idea of microstructure. 

In [ ]:
combined_data.unfold()

for i in range(5):
        pipe = Pipeline([
                ('scale', RobustScaler(with_centering=True)),
                ('reduce_dims', umap.UMAP(densmap=True, n_neighbors=10, n_components=3, min_dist=0, metric='chebyshev')),
                ])

        reduced_data = pipe.fit_transform(combined_data.data)
        clust = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=10, prediction_data=True)
        clust.fit(reduced_data)

        x,y = 50,150

        memberships_all_soft = hdbscan.all_points_membership_vectors(clust)
        memberships_highest_soft = np.array([np.max(x) for x in memberships_all_soft])
        memberships_highest_soft_spatial = memberships_highest_soft.reshape(x, y)
        labels_highest_soft = np.array([np.argmax(x) for x in memberships_all_soft])
        labels_highest_soft_spatial = labels_highest_soft.reshape(x, y)
        no_cluster_soft = len(set(labels_highest_soft))

        plot_label_map(labels_highest_soft_spatial,no_cluster_soft) 
        
        plt.title(f'{i}th run')
        plt.savefig(f'{i}th run')
        print(f'{i}th run completed')
        plt.close()

0th run completed
1th run completed
2th run completed
3th run completed
4th run completed


# Fixed random seed to directly replicate results in the paper

In [59]:
combined_data.unfold()

pipe = Pipeline([
        ('scale', RobustScaler(with_centering=True)),
        ('reduce_dims', umap.UMAP(densmap=True, n_neighbors=10, n_components=3, min_dist=0, metric='chebyshev', random_state=10)),
        ]) # random_state=10 is ok

reduced_data = pipe.fit_transform(combined_data.data)
clust = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=10, prediction_data=True)
clust.fit(reduced_data)

x,y = 50,150

memberships_all_soft = hdbscan.all_points_membership_vectors(clust)
memberships_highest_soft = np.array([np.max(x) for x in memberships_all_soft])
memberships_highest_soft_spatial = memberships_highest_soft.reshape(x, y)
labels_highest_soft = np.array([np.argmax(x) for x in memberships_all_soft])
labels_highest_soft_spatial = labels_highest_soft.reshape(x, y)
no_cluster_soft = len(set(labels_highest_soft))

plot_label_map(labels_highest_soft_spatial,no_cluster_soft) 

c:\ProgramData\Miniconda3\envs\datascience_3.9\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
